<a href="https://colab.research.google.com/github/Abhi5241/Player-ReIdentification-Using-YOLO-DeepSort/blob/main/PlayerReidentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🏃‍♂️ Player Re-ID on a Single Camera Feed

This notebook will:
- Install the tools we need  
- Let you upload your YOLOv11 model and video  
- Run detection & tracking  
- Show you the final video with persistent player IDs  


**Why?** We need:
- **Ultralytics** for YOLO detection  
- **DeepSORT** for keeping IDs consistent  
- **OpenCV** to read/write video  


In [1]:
!pip install ultralytics deep-sort-realtime opencv-python-headless


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

**Why?** Instead of Drive, we’ll upload both the `.pt` model file and your `.mp4` clip right here.  
After you run this cell, click “Choose Files” and pick your `best.pt` and `15sec_input_720p.mp4`.


In [2]:
from google.colab import files

uploaded = files.upload()   # select your .pt and .mp4 files
print("Uploaded:", list(uploaded.keys()))


Saving 15sec_input_720p.mp4 to 15sec_input_720p.mp4
Saving best.pt to best.pt
Uploaded: ['15sec_input_720p.mp4', 'best.pt']


**What’s happening here?**  
1. Load the uploaded YOLOv11 model  
2. Start DeepSORT to track each detection  
3. Read each frame, detect players, update tracks  
4. Draw colored boxes + ID labels  
5. Save to `tracked_output.mp4`


In [6]:
import cv2
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

model   = YOLO("best.pt")
tracker = DeepSort(max_age=30, n_init=3, max_cosine_distance=0.4)

cap  = cv2.VideoCapture("15sec_input_720p.mp4")
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter("tracked_output.mp4", fourcc,
                      cap.get(cv2.CAP_PROP_FPS),
                      (int(cap.get(3)), int(cap.get(4))))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 1) Detect players/ball
    res    = model(frame)[0]
    boxes  = res.boxes.xyxy.cpu().numpy()   # shape: (N,4)
    scores = res.boxes.conf.cpu().numpy()   # shape: (N,)

    # 2) Format for DeepSORT: a list of ([x1,y1,x2,y2], score)
    dets = [ (b.tolist(), float(s))
             for b, s in zip(boxes, scores) ]

    # 3) Update tracker
    tracks = tracker.update_tracks(dets, frame=frame)

    # 4) Draw boxes + IDs
    for t in tracks:
        if not t.is_confirmed():
            continue
        x1, y1, x2, y2 = map(int, t.to_ltrb())
        cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
        cv2.putText(frame, f"ID {t.track_id}",
                    (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0,255,0), 2)

    out.write(frame)

cap.release()
out.release()
print("✅ Tracking complete — see tracked_output.mp4")



0: 384x640 1 ball, 16 players, 2 referees, 2019.7ms
Speed: 3.2ms preprocess, 2019.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 18 players, 2 referees, 2117.6ms
Speed: 3.0ms preprocess, 2117.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2116.4ms
Speed: 2.5ms preprocess, 2116.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2047.6ms
Speed: 1.9ms preprocess, 2047.6ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 14 players, 2 referees, 2148.6ms
Speed: 2.3ms preprocess, 2148.6ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 ball, 16 players, 2 referees, 2088.4ms
Speed: 2.4ms preprocess, 2088.4ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 15 players, 2 referees, 2069.2ms
Speed: 3.8ms preprocess, 2069.2ms inference, 1.1ms 